# Consigne
Créez une application tkinter qui permet de dessiner des cercles sur un espace, de centre et de rayon différents. On pourra travailler sur trois versions :
* Dans une première version, le centre et le rayon pourront être indiqués avec 3 Entry, et le dessin pourra être lancé via un bouton.
* Dans un deuxième temps, il faudra plutôt permettre de placer un cercle en cliquant dans la zone de travail.
* Dans un troisième temps, l'utilisateur devra pouvoir utiliser le drag-and-drop pour spécifier le rayon du cercle. Plus précisément, pour poser un cercle sur le plan de travail, il faut cliquer dans cette zone, puis, tout en maintenant le bouton gauche appuyé, il faut déplacer le curseur de la souris. La relachement du bouton gauche de la souris indique alors le rayon, et lance le dessin.  

La taille de la zone de travail peut être fixée à 100 pixels de large et 100 pixels de hauteur. Les cercles peuvent être définis en dehors de cette zone (ce qui n'est possible que dans les versions 1 et 2).

L'application doit proposer les fonctionnalités suivantes :
* sauvegarder l'image créée sous la forme d'une image jpeg. On pourra utiliser la bibliothèque PIL pour cela.

On pourra penser optionnellement aux fonctionnalités suivantes, mais elles ne seront pas développée dans ce notebook :
* préciser la couler du cercle
* permettre une couleur de remplissage des cercles (qui deviennents des disques !)
* permettre de changer l'ordre de dessin des cercles (équivallent des "Placer devant", "Placer derrière", "Reculer", "Avancer" des outils de dessin, ou de présentation assistée par ordinateur.
* déplacer les cercles





# Modèle du monde
Nous allons commencer par définir comment nous allons décrire les élements à manipuler.
Nous devons manipuler des cercles. Il est nécessaire de mémoriser l'ensemble des cercles car il faudra redessiner tous ces cercles quand la fenêtre sera iconifiée, en partie cachée, puis de nouveau visible.

Un cercle est défini par 3 valeurs : abscisse, ordonnée, et rayon.

En Python, cela peut être représenté par un tuple :

In [32]:
cercle = (23,56,78)

Un affiche simple d'une information de ce cerle peut être :

In [33]:
print("la rayon du cercle est : ",cercle[2])

la rayon du cercle est :  78


Un affichage complet (en utilisant format) peut être :

In [34]:
print("cercle :\n   abscisse : {}\n   ordonnée : {}\n   rayon    : {}".format(cercle[0],cercle[1],cercle[2]))

cercle :
   abscisse : 23
   ordonnée : 56
   rayon    : 78


Mais, comme les deux informations "abscisse" et "ordonnée" en forme en fait une seule : "centre", je choisis de rassembler l'abscisse et l'ordonnée en un tuple et le cercle est alors défini par ce tuple plus le rayon :

In [35]:
cercle = ((23,56),78)
print("la rayon du cercle est : ",cercle[1])
print("cercle :\n   abscisse : {}\n   ordonnée : {}\n   rayon    : {}".format(cercle[0][0],cercle[0][1],cercle[1]))

la rayon du cercle est :  78
cercle :
   abscisse : 23
   ordonnée : 56
   rayon    : 78


On peut aussi penser à une produisant la description texte compacte d'un cercle :

In [36]:
def descriptionCercle(c):
    return "(c=({},{}),r={})".format(c[0][0],c[0][1],c[1])

print(descriptionCercle(cercle))

(c=(23,56),r=78)


J'ai réfléchi à ajouter une information supplémentaire au cercle : son nom. Cela aurait permis d'enrichir l'affichage ("le centre du cercle X est..."). Mais comme l'application n'a pas besoin de cette information, je ne suis pas allée plus loin dans cette direction.

L'ensemble des cercles sera stocké dans une liste Python :

In [37]:
listeCercles = [((23,56),78),((0,20),43)]

for c in listeCercles:
    print(descriptionCercle(c))


(c=(23,56),r=78)
(c=(0,20),r=43)


Mais bon, c'était juste pour vous montrer un exemple ; au lancement de l'application, il n'y a aucun cercle. Donc la liste est vide : 

In [38]:
listeCercles = []

# L'interface
L'interface va consister en
* un Canvas de 1000x1000 pour dessiner les cercles
* une ligne de trois Entry pour entrer les informations d'un cercle
* un bouton pour valider les informations, et dessiner le cercle

Commençons par créer la fenêtre qui va accueillir tout cela :

In [39]:
from tkinter import *
fen  = Tk()
fen.title("(vraiment) Petit logiciel de dessin")

''

Maintenant le Canvas :

In [40]:
fen.title("Le jeu du disque")

zoneDessin = Canvas(fen,width=800,height=800)

zoneDessin.pack()


Pour les 3 Entry, je choisis de les présenter sur une seule ligne en dessous du Canvas. Pour cela, je vais les intégrer dans une Frame. Puis je vais placer chaque Entry de gauche à droite dans cette Frame. Pour la lisibilité pour l'utilisateur, j'insère aussi dans les Frame des Label pour indiquer à quoi sert chaque Entry :

In [41]:
## La Frame englobante
f = Frame(fen) ## fen : f sera packée dans fen

## abscisse
entreeAbscisse = Entry(f) ## argument f pour spécifier que ces widgets seront "packées" dans f
labelAbscisse = Label(f,text="Abscisse : ")
labelAbscisse.pack(side=LEFT) ## LEFT : pour insertion horizontale de gauche à droite
entreeAbscisse.pack(side=LEFT)

## ordonnee
entreeOrdonnee = Entry(f)
labelOrdonnee = Label(f,text="Ordonnée : ")
labelOrdonnee.pack(side=LEFT)
entreeOrdonnee.pack(side=LEFT)

## rayon
entreeRayon = Entry(f)
labelRayon = Label(f,text="Rayon : ")
labelRayon.pack(side=LEFT)
entreeRayon.pack(side=LEFT)

## ne pas oublier de packer f
f.pack()


On s'occupe maintenant du bouton :

In [42]:
boutonInsert = Button(fen,text="Ajouter")
boutonInsert.pack()

Voilà, l'interface est prête (si ce n'est qu'elle ne fait rien). Pour vous en convaincre, vous pouvez copier coller l'ensemble du code ci-dessus dans un seul fichier et ajouter :

`fen.mainloop()`

Il reste maintenant à lier le bouton `Ajouter` à une fonction qui va lire le contenu des 3 Entry et qui va ajouter un cercle à la liste listeCercles, et qui va redessiner les cercles dans le Canvas.


# Lien entre interface et modèle

Commençons par la procédure qui dessine les cercles dans le Canvas en fonction du contenu de la liste `listeCercles`

In [43]:
def redessineCanvas(zone,listeCercles):
    zone.delete(ALL)
    for cercle in listeCercles:
        rayon = cercle[1]
        centre = cercle[0]
        xCentre = centre[0]
        yCentre = centre[1]
        ## a partir du centre et du rayon, je calcule les coordonnees haut gauche (hg) et bas droit (bd)
        ## du carre qui englobe le cercle
        xhg = xCentre-rayon
        yhg = yCentre-rayon
        xbd = xCentre+rayon
        ybd = yCentre+rayon
        zone.create_oval(xhg,yhg,xbd,ybd)

Dans la fonction ci-dessus, j'aurais pu ne pas donner d'argument à `redessineCanvas` et directement accéder aux variables `listeCercles` et `zoneDessin`. Mais une bonne pratique et d'interdire à une fonction d'accéder aux variables globales. En effet, cela permettra plus facilement, pour un futur besoin, de copier-coller cette fonction pour un autre programme, qui n'aurait pas défini en global les variables `listeCercles` et `zoneDessin`. 

Maintenant, je crée la fonction qui `ajouteCercle` qui va être associée au bouton `Ajouter`. Cette fonction ne doit prendre aucun argument. Elle va donc modifier les variables globales.

In [44]:
def ajouteCercle():
    ## lecture des 3 entrees (Entry)
    abscisseCentre = int(entreeAbscisse.get())
    ordonneeCentre = int(entreeOrdonnee.get())
    rayon = int(entreeRayon.get())
    ## ajout du cercle à la liste
    listeCercles.append(((abscisseCentre,ordonneeCentre),rayon))
    ## rafraichissement du Canvas
    redessineCanvas(zoneDessin,listeCercles)

Vous aurez remarqué que cette fonction ne vérifie pas que les entrées données par l'utilisateur sont correctes. Il faudrait vérifier que les Entry ne sont pas vides et qu'elles contiennent des valeurs entières non négatives.

Il reste à lier la fonction `ajouteCercle` au bouton :

In [45]:
boutonInsert.config(command=ajouteCercle)

Pour terminer, on lance la boucle d'écoute des événements :

In [46]:
fen.mainloop()